In [7]:
# GUARDAR OS DADOS GEOGRÁFICOS EM TIFF

In [8]:
from pathlib import Path
import os
import rasterio
import re
import numpy as np
import geopandas as gpd
from rasterio.enums import Resampling
from rasterio import features
from rasterio.warp import calculate_default_transform, reproject, Resampling
from glob import glob
import shutil
from pathlib import Path

In [9]:
aspect_path = r"../../Data/Raw/GIS_data/aspect/aspect/50m/copernicus/v2/aspect.asc"
elevation_path = r"../../Data/Raw/GIS_data/elevation/elevation/50m/copernicus/v2/elevation.asc"

last_fire_paths = list(Path(r"../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire").rglob("*.asc"))

fuel_paths = list(Path(r"../../Data/Raw/GIS_data/fuel_models/100m/portugal/PT-FireSprd").rglob("*.asc"))

GIS_paths = [aspect_path, elevation_path] + last_fire_paths + fuel_paths

print(GIS_paths)

['../../Data/Raw/GIS_data/aspect/aspect/50m/copernicus/v2/aspect.asc', '../../Data/Raw/GIS_data/elevation/elevation/50m/copernicus/v2/elevation.asc', WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2016/years_since_fire_2016.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2017/years_since_fire_2017.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2018/years_since_fire_2018.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2019/years_since_fire_2019.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2020/years_since_fire_2020.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2021/years_since_fire_2021.asc'), WindowsPath('../../Data/Raw/GIS_data/years_sin

In [10]:
for path in GIS_paths:
    path = Path(path)

    print(path)

    # substituir "Raw" por "Interim" no caminho
    tif_path = Path(str(path).replace(r"Raw", r"Interim")).with_suffix(".tif")

    print(tif_path)

    # garantir que a pasta existe
    os.makedirs(tif_path.parent, exist_ok=True)

    # só converte se ainda não existir
    if not tif_path.exists():
        with rasterio.open(path) as src:
            profile = src.profile
            profile.update(driver="GTiff")  # saída em GeoTIFF
            with rasterio.open(tif_path, "w", **profile) as dst:
                dst.write(src.read())
        print(f"✅ Convertido: {tif_path}")
    else:
        print(f"ℹ️ Já existe: {tif_path}")


..\..\Data\Raw\GIS_data\aspect\aspect\50m\copernicus\v2\aspect.asc
..\..\Data\Interim\GIS_data\aspect\aspect\50m\copernicus\v2\aspect.tif
ℹ️ Já existe: ..\..\Data\Interim\GIS_data\aspect\aspect\50m\copernicus\v2\aspect.tif
..\..\Data\Raw\GIS_data\elevation\elevation\50m\copernicus\v2\elevation.asc
..\..\Data\Interim\GIS_data\elevation\elevation\50m\copernicus\v2\elevation.tif
ℹ️ Já existe: ..\..\Data\Interim\GIS_data\elevation\elevation\50m\copernicus\v2\elevation.tif
..\..\Data\Raw\GIS_data\years_since_last_fire\years_since_last_fire\100m\portugal\isa\v2016\years_since_fire_2016.asc
..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\100m\portugal\isa\v2016\years_since_fire_2016.tif
ℹ️ Já existe: ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\100m\portugal\isa\v2016\years_since_fire_2016.tif
..\..\Data\Raw\GIS_data\years_since_last_fire\years_since_last_fire\100m\portugal\isa\v2017\years_since_fire_2017.asc
..\..\Data\Interim\GIS_data\years

In [11]:
# Caminhos
base_50m = Path("../../Data/Interim/GIS_data/years_since_last_fire/years_since_last_fire/50m/portugal/isa/")
base_100m = Path("../../Data/Interim/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/")
dest = Path("../../Data/Interim/GIS_data/years_since_last_fire/PT-FireSprd")

# Criar diretório destino se não existir
dest.mkdir(parents=True, exist_ok=True)

# Anos alvo
anos = [str(a) for a in range(2015, 2026)]

# Expressão regular para extrair ano
re_ano = re.compile(r"(20\d{2})")

def listar_tiffs(base_path):
    resultados = {}
    for root, _, files in os.walk(base_path):
        for f in files:
            if f.lower().endswith(".tif"):
                match = re_ano.search(f)
                if match:
                    ano = match.group(1)
                    if ano in anos and ano not in resultados:
                        resultados[ano] = Path(root) / f
    return resultados

# Recolher ficheiros
tiffs_50m = listar_tiffs(base_50m)
tiffs_100m = listar_tiffs(base_100m)

# Combinar com prioridade 50m
selecionados = {}
for ano in anos:
    if ano in tiffs_50m:
        selecionados[ano] = tiffs_50m[ano]
    elif ano in tiffs_100m:
        selecionados[ano] = tiffs_100m[ano]

# Copiar ficheiros
for ano, src in selecionados.items():
    destino = dest / src.name
    shutil.copy2(src, destino)
    print(f"✅ Copiado {src} → {destino}")

print("\n🎯 Concluído! Tiffs copiados para:", dest)


✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2015\years_since_fire_2015.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2015.tif
✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2016\years_since_fire_2016.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2016.tif
✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2017\years_since_fire_2017.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2017.tif
✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2018\years_since_fire_2018.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2018.tif
✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2019\years_since

In [12]:
import os
import re
import numpy as np
from glob import glob

import geopandas as gpd
import rasterio
from rasterio import features
from rasterio.warp import calculate_default_transform, reproject, Resampling

from dask.distributed import Client, LocalCluster, progress

# =========================
# CONFIGURAÇÃO
# =========================
tiff_dir = os.path.normpath("../../Data/Interim/GIS_data/years_since_last_fire/PT-FireSprd")
COS_2018_PATH = os.path.normpath("../../Data/Raw/GIS_data/COS/COS2018v3-S2-shp/COS2018v3-S2.shp")
COS_2023_PATH = os.path.normpath("../../Data/Raw/GIS_data/COS/COS2023v1-S2-shp/COS2023v1-S2.shp")
TARGET_CRS = "EPSG:32629"

# =========================
# DASK CLUSTER OTIMIZADO
# =========================
cluster = LocalCluster(
    n_workers=8,          # 8 workers → ótimo para 128GB RAM
    threads_per_worker=1, # GDAL + NumPy preferem 1 thread
    memory_limit="16GB"   # cada worker tem 16GB
)
client = Client(cluster)
print(client)

# =========================
# FUNÇÕES AUXILIARES
# =========================

def preparar_cos_local(cos_path):
    """Lê e filtra COS dentro do worker"""
    gdf = gpd.read_file(cos_path)
    field = next(c for c in gdf.columns if c.lower().startswith("cos"))
    gdf = gdf[gdf[field].astype(str).str.match(r"^[345]")]
    if gdf.crs != TARGET_CRS:
        gdf = gdf.to_crs(TARGET_CRS)
    return gdf

def process_tiff(tiff_path):
    """Processa 1 TIFF: reprojetar, rasterizar máscara de vegetação e salvar"""
    try:
        # Determinar ano e COS
        year = int(re.search(r"(\d{4})", tiff_path).group(1))
        cos_path = COS_2018_PATH if year <= 2020 else COS_2023_PATH
        gdf = preparar_cos_local(cos_path)

        tmp_path = tiff_path.replace(".tif", "_utm29n_tmp.tif")

        # Reprojetar se necessário
        with rasterio.open(tiff_path) as src:
            if src.crs != TARGET_CRS:
                transform, width, height = calculate_default_transform(
                    src.crs, TARGET_CRS, src.width, src.height, *src.bounds
                )
                meta = src.meta.copy()
                meta.update(crs=TARGET_CRS, transform=transform, width=width, height=height)

                with rasterio.open(tmp_path, "w", **meta) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            rasterio.band(src, i),
                            rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=TARGET_CRS,
                            resampling=Resampling.nearest
                        )
                path = tmp_path
            else:
                path = tiff_path

        # Abrir raster e aplicar máscara de vegetação
        with rasterio.open(path) as burned_src:
            burned = burned_src.read(1).astype(np.float32)
            nodata = burned_src.nodata
            burned[burned == nodata] = np.nan

            veg_mask = features.rasterize(
                [(geom, 1) for geom in gdf.geometry],
                out_shape=(burned_src.height, burned_src.width),
                transform=burned_src.transform,
                fill=0,
                dtype=np.uint8
            ).astype(bool)

            burned[np.isnan(burned) & veg_mask] = 0
            burned = np.nan_to_num(burned, nan=nodata if nodata is not None else 0)

            out_path = tiff_path.replace(".tif", "_p.tif")
            meta = burned_src.meta.copy()
            meta.update(dtype=rasterio.float32, crs=TARGET_CRS)

            with rasterio.open(out_path, "w", **meta) as dst:
                dst.write(burned, 1)

        if os.path.exists(tmp_path):
            os.remove(tmp_path)

        return out_path

    except Exception as e:
        return f"ERRO {os.path.basename(tiff_path)}: {e}"

# =========================
# PIPELINE PRINCIPAL
# =========================

tiff_files = [f for f in glob(os.path.join(tiff_dir, "**", "*.tif"), recursive=True)
              if "_p" not in f]

print(f"\n🚀 Processando {len(tiff_files)} TIFFs em paralelo com Dask...")

# Submeter tasks para Dask
futures = [client.submit(process_tiff, f) for f in tiff_files]

# Barra de progresso
progress(futures)

# Coletar resultados
results = client.gather(futures)

print("\n✅ Ficheiros processados:")
for r in results:
    print(" ", r)


c:\Conda\envs\firehack\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59752 instead
  warnings.warn(
c:\Conda\envs\firehack\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.78s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


<Client: 'tcp://127.0.0.1:59753' processes=8 threads=8, memory=119.21 GiB>

🚀 Processando 11 TIFFs em paralelo com Dask...

✅ Ficheiros processados:
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2015_p.tif
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2016_p.tif
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2017_p.tif
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2018_p.tif
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2019_p.tif
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2020_p.tif
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2021_p.tif
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2022_p.tif
  ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2023_p.tif
  ..\..\Data\Interim\GI